**Import Required Libraries and Set Up the Environment**

In this cell, we import necessary libraries from Qiskit, Quairkit, NumPy, and other dependencies. These imports include tools for building and manipulating quantum circuits, handling noise models, and utilizing IBM's quantum runtime service.



Before running this Jupyter Notebook, please ensure that the following Python packages are installed. You can use the following pip command to install these packages:

```sh
pip install quairkit qiskit qiskit_ibm_runtime qiskit_aer tqdm pylatexenc matplotlib
```

In [ ]:
import os

# Type aliases for better readability and clarity of type annotations
from typing import Optional, Tuple

# Import required libraries
import numpy as np  # NumPy for numerical operatiQuAIRKitmatrices and state vectors
from numpy import ndarray

# Import Qiskit components for building and simulating quantum circuits
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import CRYGate  # Controlled rotation around the Y-axis
from qiskit.quantum_info import DensityMatrix, Statevector  # Quantum state manipulations

# Import Qiskit Aer simulator and noise model components
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

# Import IBM Runtime Service to interact with IBM Quantum devices
from qiskit_ibm_runtime import QiskitRuntimeService

# Import the partial_trace and random_unitary functions from quairkit's quantum information tools
from quairkit.qinfo import partial_trace
from quairkit.database import random_unitary

# Import TQDM for creating progress bars during simulations
from tqdm import tqdm


**Create the Ansatz Circuit V1: `create_VCG_2()`**

In this cell, we define a function `create_VCG_2()` that constructs a quantum circuit named V1, which includes a series of advanced gate operations. This circuit serves as an ansatz for quantum algorithms and consists of Toffoli gates, controlled rotations, and swaps between qubits.

**Function Details**:
1. **Input and Initialization**:
   - The function initializes a quantum circuit with 3 qubits.
   
2. **Gate Operations**:
   - **Toffoli Gate (CCX)**: The function first applies a Toffoli (controlled-controlled-X) gate with qubits 0 and 1 as controls and qubit 2 as the target.
   - **CNOT Gate**: A controlled-NOT gate is applied from qubit 1 (control) to qubit 0 (target).
   - **Pauli-X Gates**: The function applies an X gate to qubit 2, and later in the circuit, to qubits 0, 1, and 2 at different stages.
   - **Controlled Rotation (CRY)**: The function then applies a controlled-Y rotation gate (`CRY`) with an angle of `π/2`, acting between qubits 2 (control) and 1 (target).
   - **Double-Controlled Rotation (CCRY)**: A doubly-controlled RY gate is appended, using the `CRYGate` with additional control. This operation involves qubits 2 and 0 as controls and qubit 1 as the target.
   - **Swap Gates**: Finally, swap operations are applied to change the positions of qubits in a sequential manner (between qubits 0 and 1, then between 1 and 2).

**Purpose**:
This ansatz circuit is designed to be flexible and potentially used in variational quantum algorithms. It includes a diverse set of quantum gates, making it suitable for exploring complex state transformations.


In [ ]:
def create_VCG_2() -> QuantumCircuit:
    """
    Create ansatz of V1 using Qiskit.
    
    This function constructs a quantum circuit with 3 qubits, involving various gate operations 
    such as Toffoli, CNOT, Pauli-X, CRY (controlled rotation around Y-axis), and swap gates.

    Returns:
        qc: The constructed QuantumCircuit object.
    """
    # Initialize a quantum circuit with 3 qubits
    qc = QuantumCircuit(3, name="VCG_2")

    # Step 1: Apply Toffoli gate (CCX gate) with qubits 0 and 1 as controls, and qubit 2 as target
    qc.ccx(0, 1, 2)

    # Step 2: Apply CNOT gate, with qubit 1 as control and qubit 0 as target
    qc.cx(1, 0)

    # Step 3: Apply X gate to qubit 2 to flip its state
    qc.x(2)

    # Step 4: Apply Controlled-Y rotation (CRY) gate with π/2 rotation, with qubit 2 as control and qubit 1 as target
    qc.cry(np.pi / 2, 2, 1)

    # Step 5: Apply X gate to qubit 0 to flip its state
    qc.x(0)

    # Step 6: Add a doubly-controlled CRY gate with a rotation of π/2
    # This gate has two control qubits (qubits 2 and 0) and one target qubit (qubit 1)
    cry_controlled = CRYGate(np.pi / 2).control(1)
    qc.append(cry_controlled, [2, 0, 1])

    # Step 7: Apply X gates to qubits 0 and 2 to flip their states
    qc.x([0, 2])

    # Step 8: Apply X gate to qubit 1, followed by a CNOT from qubit 1 to qubit 0, then an X gate again to qubit 1
    qc.x(1)
    qc.cx(1, 0)
    qc.x(1)

    # Step 9: Swap qubits to change their positions
    # First, swap qubits 0 and 1, then swap qubits 1 and 2
    qc.swap(0, 1)
    qc.swap(1, 2)

    # Return the constructed quantum circuit
    return qc


**Create the Ansatz Circuit V2: `create_VCG_3()`**

In this cell, we define a function `create_VCG_3()` that constructs a quantum circuit named V2, involving a combination of controlled and multi-controlled operations, Toffoli gates, and swap operations. This circuit is useful as an ansatz for variational quantum algorithms, representing a complex set of transformations involving multiple qubits.

**Function Details**:
1. **Initialization**:
   - A quantum circuit with 4 qubits is initialized.

2. **Gate Operations**:
   - **Pauli-X Gates and CNOT**:
     - Apply an X gate to qubit 0, followed by a CNOT between qubit 0 (control) and qubit 2 (target), and then another X gate to qubit 0.
   - **Toffoli (CCX) and CNOT Gates**:
     - Apply a Toffoli gate (`CCX`) between qubits 3 and 2 as controls and qubit 1 as target.
     - Apply additional CNOT gates between qubits 3 and 2, and between qubits 1 and 2.
   - **Controlled-Y Rotation Gates**:
     - Define `pi` and `theta` for subsequent rotation angles.
     - Apply a controlled-Y (`CRY`) gate with rotation angle `π` between qubits 1 (control) and 3 (target).
   - **Multi-Controlled RY Gates**:
     - Using the `ccry()` function, create and append a doubly-controlled Y rotation gate with angle `-theta` and then with `theta`. These operations involve three control qubits and one target qubit.
   - **Additional Gates and Swaps**:
     - Apply CNOT, swap, and Pauli-X gates to change the state and arrangement of the qubits.
     - Use `CSWAP` and `CCX` gates for conditional and multi-control logic.

The constructed quantum circuit includes a diverse set of quantum operations, making it suitable for implementing complex quantum algorithms and exploring entangled quantum states.


In [ ]:
def create_VCG_3() -> QuantumCircuit:
    """
    Create ansatz of V2 using Qiskit.
    
    This function constructs a quantum circuit with 4 qubits, involving multiple quantum gate operations 
    such as X gates, CNOT gates, Toffoli gates, multi-controlled rotations, and swaps. This ansatz is 
    useful in exploring complex state preparations for variational quantum algorithms.

    Returns:
        qc: The constructed QuantumCircuit object.
    """
    # Initialize a quantum circuit with 4 qubits
    qc = QuantumCircuit(4, name="VCG_3")

    # Step 1: Apply X gate to qubit 0, then CNOT between qubit 0 and qubit 2, and X gate to qubit 0 again
    qc.x(0)
    qc.cx(0, 2)
    qc.x(0)

    # Step 2: Apply Toffoli gate (CCX gate) between qubits 3 and 2 as controls, and qubit 1 as target
    qc.ccx(3, 2, 1)

    # Step 3: Apply CNOT gates between qubits 3 and 2, then between qubits 1 and 2
    qc.cx(3, 2)
    qc.cx(1, 2)

    # Step 4: Define pi and theta for controlled Y rotations
    pi = np.pi
    theta = 2 * np.arccos(np.sqrt(2 / 3))

    # Step 5: Apply controlled-Y (CRY) rotation with π rotation between qubit 1 and qubit 3
    qc.x(1)
    qc.cry(pi, 1, 3)

    # Step 6: Apply a doubly-controlled CRY gate with negative theta, followed by positive theta
    # Adding two controls makes it a triple-controlled gate
    ccry_neg_theta = CRYGate(-theta).control(1).control(1)
    qc.append(ccry_neg_theta, [0, 1, 2, 3])  # Apply triple-controlled CRY with negative theta

    # Reset X gate on qubit 1 to prepare for next multi-control gate
    qc.x(1)

    # Apply triple-controlled CRY gate with positive theta
    ccry_theta = CRYGate(theta).control(1).control(1)
    qc.append(ccry_theta, [0, 1, 2, 3])  # Apply triple-controlled CRY with positive theta

    # Step 7: Apply CNOT gate between qubits 3 and 0, then apply swap between qubits 1 and 2
    qc.cx(3, 0)
    qc.swap(1, 2)

    # Step 8: Apply X gates to qubits 0 and 1 to flip their states
    qc.x([0, 1])

    # Step 9: Apply CNOT gate from qubit 0 to qubit 1, followed by a CSWAP and a CCX gate
    qc.cx(0, 1)
    qc.cswap(0, 1, 2)
    qc.ccx(0, 1, 2)

    # Return the constructed quantum circuit
    return qc


**Create the Ansatz Circuit V1: `create_V1()`**

In this cell, we define a function `create_V1()` that constructs a quantum circuit involving seven qubits. This circuit incorporates a combination of gate operations, sub-circuits (`VCG_2` and `VCG_3`), controlled gates, and swap operations. This ansatz can be used in advanced quantum algorithms, where complex interactions between qubits are needed.

**Function Details**:
1. **Initialization**:
   - A quantum circuit with 7 qubits is initialized.

2. **Gate Operations**:
   - **Swap Gates**:
     - Qubits 2 and 5 are swapped to modify their positions within the circuit.
   - **Sub-Circuit Composition (`VCG_2`)**:
     - The circuit `VCG_2` is composed into `V1`, using qubits 0, 1, and 2 as the target qubits.
   - **Controlled NOT (CNOT)**:
     - A CNOT gate is applied between qubit 0 (control) and qubit 6 (target) to create entanglement between these qubits.
   - **Sub-Circuit Composition (`VCG_3` Inverse)**:
     - The inverse of the `VCG_3` circuit is composed using qubits 3, 4, 5, and 6. This is used to reverse the transformations of `VCG_3` and create interference effects.
   - **Additional Swap Operations**:
     - Qubits 3 and 6 are swapped, followed by a swap between qubits 1 and 3. These operations are used to modify the layout of qubits to achieve a specific configuration, aligning with the logic of `quairkit`.

This circuit includes a combination of direct gate operations and the composition of previously defined sub-circuits, making it suitable for use in more complex quantum algorithms and state preparation.


In [ ]:
def create_V1():
    V1 = QuantumCircuit(7)
    
    # Swapping qubits 2 and 5
    V1.swap(2, 5)
    
    # Composing the VCG_2 circuit
    VCG_2_circuit = create_VCG_2()
    V1.compose(VCG_2_circuit, qubits=[0, 1, 2], inplace=True)
    
    # Applying CNOT between qubits 0 and 6
    V1.cx(0, 6)
    
    # Composing the inverse of the VCG_3 circuit
    VCG_3_circuit = create_VCG_3()
    VCG_3_inverse = VCG_3_circuit.inverse()
    V1.compose(VCG_3_inverse, qubits=[3, 4, 5, 6], inplace=True)
    
    # Additional swaps to match the quairkit logic
    V1.swap(3, 6)
    V1.swap(1, 3)
    
    return V1

**Create the Ansatz Circuit V2: `create_V2()`**

This function creates a quantum circuit named V2, involving seven qubits. The circuit includes a combination of swap operations, controlled gates, and sub-circuit compositions (`VCG_3` and `VCG_2`). This ansatz is designed to apply complex transformations involving multiple qubits, making it suitable for advanced quantum algorithms and experiments.

**Function Details**:
1. **Initialization**:
   - A quantum circuit with 7 qubits is initialized.

2. **Gate Operations**:
   - **Swap Gates**:
     - Swaps are used to rearrange the qubits, first between qubits 1 and 3, and later between other pairs like 4 and 6, and 5 and 6.
   - **Sub-Circuit Composition (`VCG_3`)**:
     - The `VCG_3` circuit is composed using qubits 0, 1, 2, and 3.
   - **Controlled NOT (CNOT)**:
     - A CNOT gate is applied between qubit 4 (control) and qubit 3 (target) to introduce entanglement.
   - **Inverse of `VCG_2`**:
     - The inverse of the `VCG_2` circuit is composed into the current circuit, involving qubits 4, 5, and 6.
   - **Additional Swap Operations**:
     - Further swaps are used to align the circuit to a particular layout that is consistent with a specific logic referred to as "Avocado."

The constructed circuit is intended to provide a flexible and powerful ansatz that can be used in variational quantum algorithms and simulations.


In [ ]:
def create_V2() -> QuantumCircuit:
    """
    Create ansatz V2 using Qiskit.
    
    This function constructs a quantum circuit with 7 qubits, involving a combination of swap gates,
    sub-circuit compositions, controlled gates, and additional swap operations. The constructed circuit
    includes the sub-circuits VCG_3 and the inverse of VCG_2.

    Returns:
        V2: The constructed QuantumCircuit object representing ansatz V2.
    """
    # Initialize a quantum circuit with 7 qubits
    V2 = QuantumCircuit(7, name="V2")

    # Step 1: Swap qubits 1 and 3 to rearrange positions
    V2.swap(1, 3)

    # Step 2: Compose the VCG_3 circuit using qubits 0, 1, 2, and 3
    VCG_3_circuit = create_VCG_3()  # Assume VCG_3 is defined elsewhere and returns a QuantumCircuit
    V2.compose(VCG_3_circuit, qubits=[0, 1, 2, 3], inplace=True)

    # Step 3: Swap qubits 4 and 6 to rearrange positions
    V2.swap(4, 6)

    # Step 4: Apply a CNOT gate between qubits 4 (control) and 3 (target) to create entanglement
    V2.cx(4, 3)

    # Step 5: Swap qubits 5 and 6
    V2.swap(5, 6)

    # Step 6: Compose the inverse of the VCG_2 circuit using qubits 4, 5, and 6
    VCG_2_circuit = create_VCG_2()  # Assume VCG_2 is defined elsewhere and returns a QuantumCircuit
    VCG_2_inverse = VCG_2_circuit.inverse()
    V2.compose(VCG_2_inverse, qubits=[4, 5, 6], inplace=True)

    # Step 7: Additional swap operations to match the Avocado logic
    V2.swap(2, 4)
    V2.swap(1, 5)
    V2.swap(0, 4)

    # Return the constructed quantum circuit
    return V2


**Initialize Bell States and Random Statevectors**

In this cell, we define and initialize specific quantum states using Qiskit's `Statevector` class. The purpose is to prepare standard entangled states (Bell states) and other quantum states that can be used as inputs in quantum algorithms.

**Key States Defined**:
1. **psi_in** (Bell State):
   - This state is an equal superposition of the two-qubit states |00⟩ and |11⟩.
   - It represents a maximally entangled Bell state, often used in quantum communication and teleportation protocols.

2. **phi_minus** (Bell State):
   - This state is an equal superposition of the two-qubit states |01⟩ and |10⟩, with a negative relative phase between them.
   - Similar to `psi_in`, this Bell state is maximally entangled and demonstrates different phase relationships between the qubits.

These states are important building blocks in many quantum algorithms and protocols, and initializing them correctly is crucial for successful quantum experiments.


In [ ]:
# Initialize a Bell state (|ψ+⟩), which is an equal superposition of |00⟩ and |11⟩
psi_in: Statevector = (
    Statevector.from_label("00") + Statevector.from_label("11")
) / np.sqrt(2)

# Initialize another Bell state (|ϕ-⟩), which is an equal superposition of |01⟩ and |10⟩ with a negative phase
phi_minus: Statevector = (
    Statevector.from_label("01") - Statevector.from_label("10")
) / np.sqrt(2)


**Create Quantum Circuit with Specified Unitary: `create_circuit_with_unitary()`**

This function creates a quantum circuit, applying a given unitary matrix to a subset of qubits in combination with other operations. The circuit is built with 8 qubits and uses a combination of initialization, unitary transformations, and pre-defined sub-circuits (`V1` and `V2`) to achieve a specific ansatz structure.

**Function Details**:
1. **Initialization**:
   - **Input Parameters**:
     - `u`: A 2x2 unitary matrix representing a single-qubit operation.
     - `psi_in_vector`: A `Statevector` object that is used to initialize qubits 0 and 7.
   - **Quantum Register**:
     - A quantum circuit with 8 qubits is created to accommodate the operations.

2. **Quantum State Initialization**:
   - Qubits 0 and 7 are initialized with `psi_in_vector` to create the desired initial state.
   - Qubits 1 and 2 are initialized with `phi_minus` to represent another entangled Bell state.
   - Qubits 3, 4, 5, and 6 are initialized to the state |0⟩.

3. **Circuit Construction (Repeated Twice)**:
   - **Unitary Application**: The provided unitary (`u`) is applied to qubit 1.
   - **Compose V1**: The pre-defined sub-circuit `V1` is then composed into the current circuit.
   - **Unitary Reapplication**: The same unitary (`u`) is applied again to qubit 1.
   - **Compose V2**: The pre-defined sub-circuit `V2` is composed into the current circuit.

This ansatz combines unitary transformations with additional composed sub-circuits (`V1` and `V2`), making it suitable for complex quantum simulations and variational quantum algorithms.


In [ ]:
def create_circuit_with_unitary(u, psi_in_vector: Statevector) -> QuantumCircuit:
    """
    Create a quantum circuit based on the input unitary and initialized state vectors.

    Args:
        u: A unitary matrix (as a NumPy array or Qiskit Operator) to be applied as a single-qubit operation.
        psi_in_vector: A Statevector representing the initial state of qubits 0 and 7.

    Returns:
        QuantumCircuit: The constructed quantum circuit with the specified unitary applied.
    """
    # Initialize an 8-qubit quantum circuit
    qc = QuantumCircuit(8, name="unitary_ansatz")

    # Step 1: Initialize specific qubits to the desired states
    # Initialize qubits 0 and 7 with the given state vector
    qc.initialize(psi_in_vector, [0, 7])

    # Initialize qubits 1 and 2 with the Bell state |ϕ-⟩ (previously defined)
    qc.initialize(phi_minus, [1, 2])

    # Initialize qubits 3, 4, 5, and 6 to the |0> state
    for i in range(4):
        qc.initialize([1, 0], i + 3)

    # Step 2: Construct the circuit by repeating the block twice
    for _ in range(2):
        # Apply the given unitary 'u' to qubit 1
        qc.unitary(u, [1], label="U_in")

        # Compose the V1 circuit (assumed to be defined and returns a QuantumCircuit)
        V1_circuit = create_V1()  # V1 should be defined elsewhere
        qc.compose(V1_circuit, inplace=True)

        # Apply the given unitary 'u' to qubit 1 again
        qc.unitary(u, [1], label="U_in")

        # Compose the V2 circuit (assumed to be defined and returns a QuantumCircuit)
        V2_circuit = create_V2()  # V2 should be defined elsewhere
        qc.compose(V2_circuit, inplace=True)

    # Return the constructed quantum circuit
    return qc


**Initialize Bell States and Random Statevectors**

In this cell, we define and initialize specific quantum states using Qiskit's `Statevector` class. The purpose is to prepare standard entangled states (Bell states) and other quantum states that can be used as inputs in quantum algorithms.

**Key States Defined**:
1. **psi_in** (Bell State):
   - This state is an equal superposition of the two-qubit states |00⟩ and |11⟩.
   - It represents a maximally entangled Bell state, often used in quantum communication and teleportation protocols.

2. **phi_minus** (Bell State):
   - This state is an equal superposition of the two-qubit states |01⟩ and |10⟩, with a negative relative phase between them.
   - Similar to `psi_in`, this Bell state is maximally entangled and demonstrates different phase relationships between the qubits.

These states are important building blocks in many quantum algorithms and protocols, and initializing them correctly is crucial for successful quantum experiments.


In [ ]:
def get_output_density_matrix(
    qc: QuantumCircuit,
    noise_model: Optional[NoiseModel] = None,
) -> Tuple[ndarray, int]:
    """
    Simulate a quantum circuit with the given noise model and return the final density matrix and circuit depth.

    Args:
        qc: The QuantumCircuit object to be simulated.
        noise_model: The noise model to apply during the simulation. If None, no noise will be applied.

    Returns:
        tuple: A tuple containing:
            - The final density matrix as a Qiskit DensityMatrix object.
            - The depth of the transpiled circuit.
    """
    # Step 1: Create a noisy simulator using the provided noise model (if any)
    sim_noise = AerSimulator(noise_model=noise_model)

    # Step 2: Transpile the quantum circuit for the noisy simulator to optimize for execution
    transpiled_circuit = transpile(qc, sim_noise)

    # Step 3: Add an instruction to save the density matrix of the final quantum state
    transpiled_circuit.save_density_matrix(label="output_density_matrix")

    # Step 4: Run the simulation with the noisy simulator
    simulation_result = sim_noise.run(transpiled_circuit).result()

    # Step 5: Extract the output density matrix from the simulation results and reverse the qubit ordering if needed
    output_density_matrix = simulation_result.data(0)["output_density_matrix"].reverse_qargs()

    # Return the output density matrix and the depth of the transpiled circuit
    return output_density_matrix, transpiled_circuit.depth()


**Initialize Bell States and Random Statevectors**

In this cell, we define and initialize specific quantum states using Qiskit's `Statevector` class. The purpose is to prepare standard entangled states (Bell states) and other quantum states that can be used as inputs in quantum algorithms.

**Key States Defined**:
1. **psi_in** (Bell State):
   - This state is an equal superposition of the two-qubit states |00⟩ and |11⟩.
   - It represents a maximally entangled Bell state, often used in quantum communication and teleportation protocols.

2. **phi_minus** (Bell State):
   - This state is an equal superposition of the two-qubit states |01⟩ and |10⟩, with a negative relative phase between them.
   - Similar to `psi_in`, this Bell state is maximally entangled and demonstrates different phase relationships between the qubits.

These states are important building blocks in many quantum algorithms and protocols, and initializing them correctly is crucial for successful quantum experiments.


In [ ]:
def calculate_fidelity(
    output_state_density_matrix: DensityMatrix, u: ndarray, psi_in_vector: Statevector
) -> float:
    """
    Calculate the fidelity of a given statevector with respect to an expected state,
    after applying a unitary transformation.

    Args:
        output_state_density_matrix: The density matrix representing the final state of the quantum circuit.
        u: A unitary matrix (as a NumPy array) that is applied during the circuit.
        psi_in_vector: The initial statevector before the unitary and circuit transformations.

    Returns:
        float: The calculated fidelity value, indicating how closely the output state matches the expected state.
    """
    # Step 1: Compute the partial trace over qubits not in [2, 7]
    # Retain only qubits 2 and 7 for fidelity calculation
    output_psi_dm = partial_trace(
        output_state_density_matrix.data,
        [i for i in range(output_state_density_matrix.num_qubits) if i not in [2, 7]],
    )

    # Step 2: Convert the initial statevector to an array for calculation
    psi_in_ket = psi_in_vector.data.reshape([-1, 1])

    # Step 3: Expand the unitary matrix to the larger Hilbert space using the Kronecker product
    u_expanded = np.kron(u, np.eye(2))

    return (
        psi_in_ket.conj().T
        @ u_expanded
        @ output_psi_dm.data
        @ u_expanded.conj().T
        @ psi_in_ket
    ).real.item()


**Initialize IBM Quantum Service and Load Unitary Set**

In this cell, we initialize the IBM Quantum service to access the quantum backends available on the IBM Cloud. We specify the names of the available quantum devices (`ibm_osaka`, `ibm_brisbane`, and `ibm_sherbrooke`) to be used for executing our quantum circuit. The function also loads a set of precomputed unitary matrices, which will be used during the quantum experiments.

**Key Steps**:
1. **Backend Initialization**: Define a list of IBM Quantum backends that will be used to run the quantum experiments.
2. **Service Initialization**: Initialize the IBM Quantum service. Note that sensitive credentials like `token` and `instance` are placeholders and should be securely managed, such as using environment variables.
3. **Unitary Set Loading**: Load a set of random unitary matrices from a file, which will be used during the experiment. These unitary operations are generated beforehand and used as part of the circuit.
4. **Output File Preparation**: Prepare a filename for saving the results, including the noise data collected during the experiments.

**Security Note**: Avoid including sensitive information such as API tokens in publicly accessible scripts. Use environment variables or secure vaults to manage them securely.


In [ ]:
# Define the name of the quantum circuit
circuit_name = "previous"

# List of backend names representing the available IBM Quantum devices
backends = ["ibm_osaka", "ibm_brisbane", "ibm_sherbrooke"]

# Initialize the IBM Quantum service to access the quantum devices on IBM Cloud
# Ensure to replace 'your_ibm_cloud_token_here' and 'your_instance_here' with actual values in a secure manner
service = QiskitRuntimeService(
    channel="ibm_cloud",
    token="your_ibm_cloud_token_here",  # Replace with your IBM Quantum token
    instance="your_instance_here"       # Replace with your IBM Quantum instance string
)

# Set the number of unitary operations to be tested and load the unitary set from a file
num_test = 100
# Check if the unitary set file exists
unitary_file = f"{num_test}_random_unitaries.npy"
if os.path.exists(unitary_file):
    unitary_set = np.load(unitary_file)
else:
    unitary_set = random_unitary(num_test).numpy()
    np.save(unitary_file, unitary_set)

# Prepare a filename to save the noise table from the quantum experiments
table_file_name = f"ibm_noise_table_{num_test}.csv"


**Initialize Bell States and Random Statevectors**

In this cell, we define and initialize specific quantum states using Qiskit's `Statevector` class. The purpose is to prepare standard entangled states (Bell states) and other quantum states that can be used as inputs in quantum algorithms.

**Key States Defined**:
1. **psi_in** (Bell State):
   - This state is an equal superposition of the two-qubit states |00⟩ and |11⟩.
   - It represents a maximally entangled Bell state, often used in quantum communication and teleportation protocols.

2. **phi_minus** (Bell State):
   - This state is an equal superposition of the two-qubit states |01⟩ and |10⟩, with a negative relative phase between them.
   - Similar to `psi_in`, this Bell state is maximally entangled and demonstrates different phase relationships between the qubits.

These states are important building blocks in many quantum algorithms and protocols, and initializing them correctly is crucial for successful quantum experiments.


In [ ]:
# Initialize lists to store fidelities and depths for each backend
backend_fidelities = []
backend_depths = []

# Loop over each backend to compute the average fidelity and circuit depth
for backend_name in tqdm(backends, desc="Processing backends"):

    # # Retrieve the backend using the IBM Quantum service
    # backend = service.backend(backend_name)

    # # Generate a noise model based on the characteristics of the backend
    # noise_model = NoiseModel.from_backend(backend)
    noise_model = None  # No noise model for ideal simulation

    fidelities = []  # To store fidelities for each unitary operation
    depths = []  # To store circuit depths for each unitary operation

    # Loop over each unitary matrix in the precomputed set
    for u in tqdm(unitary_set, desc=f"Processing unitary set for backend {backend_name}"):
        # Create a quantum circuit with the given unitary and initial state
        qc = create_circuit_with_unitary(u, psi_in)

        # Simulate the circuit with the noise model and retrieve the density matrix and circuit depth
        output_state_density_matrix, depth = get_output_density_matrix(qc, noise_model)

        # Calculate the fidelity of the resulting quantum state with respect to the expected state
        fidelity = calculate_fidelity(output_state_density_matrix, u, psi_in)

        # Append the calculated fidelity and circuit depth to the respective lists
        fidelities.append(fidelity)
        depths.append(depth)

    # Calculate the average fidelity and depth for the current backend
    average_fidelity = np.mean(fidelities)
    average_depth = np.mean(depths)

    # Store the average fidelity and depth for the current backend
    backend_fidelities.append(average_fidelity)
    backend_depths.append(average_depth)

    # Create a dictionary with the results for the current backend
    qc_backend_fidelity = {
        "circuit_name": circuit_name,
        "backend_name": backend_name,
        "average_fidelity": average_fidelity,
        "average_depth": average_depth,
    }

    # Save the results to the CSV file, appending the results
    with open(table_file_name, "a") as file:
        if file.tell() == 0:  # If the file is empty, write the header
            file.write("circuit_name,backend_name,average_fidelity,average_depth\n")
        file.write(
            f"{qc_backend_fidelity['circuit_name']},"
            f"{qc_backend_fidelity['backend_name']},"
            f"{qc_backend_fidelity['average_fidelity']},"
            f"{qc_backend_fidelity['average_depth']}\n"
        )

    # Alternatively, using pandas to append the results to the CSV file
    # pd.DataFrame([qc_backend_fidelity]).to_csv(table_file_name, index=False, mode="a", header=not pd.read_csv(table_file_name).shape[0])


**Visualize the Quantum Circuit**

In this cell, we visualize the quantum circuit using Qiskit's built-in drawing capabilities. The output is generated as a Matplotlib figure (`mpl`), which allows for easy integration into reports and visual analysis. This visualization provides an intuitive way to understand the quantum gates and their arrangement within the circuit.


In [ ]:
# Visualize the quantum circuit using Qiskit's Matplotlib drawer
qc.draw(output="mpl")


**Calculate the Depth of the Quantum Circuit**

In this cell, we calculate the depth of the quantum circuit using the `depth()` method. The depth of a quantum circuit is defined as the maximum number of quantum gates that are applied sequentially, one after the other. This metric is important for assessing the complexity and efficiency of the circuit, as lower depth is typically preferred for reducing decoherence and improving fidelity on noisy quantum devices.


In [ ]:
# Calculate the depth of the quantum circuit
circuit_depth = qc.depth()
circuit_depth
